**토픽 모델링(Topic Modeling)**

토픽(Topic)은 한국어로는 주제라고 합니다. 토픽 모델링(Topic Modeling)이란 기계 학습 및 자연어 처리 분야에서 토픽이라는 문서 집합의 추상적인 주제를 발견하기 위한 통계적 모델 중 하나로, 텍스트 본문의 숨겨진 의미 구조를 발견하기 위해 사용되는 텍스트 마이닝 기법입니다.



# 1. 잠재 의미 분석(Latent Semantic Analysis, LSA)
LSA는 정확히는 토픽 모델링을 위해 최적화 된 알고리즘은 아니지만, 토픽 모델링이라는 분야에 아이디어를 제공한 알고리즘이라고 볼 수 있습니다. 이에 토픽 모델링 알고리즘인 LDA에 앞서 배워보도록 하겠습니다. 뒤에서 배우게 되는 LDA는 LSA의 단점을 개선하여 탄생한 알고리즘으로 토픽 모델링에 보다 적합한 알고리즘입니다.

BoW에 기반한 DTM이나 TF-IDF는 기본적으로 단어의 빈도 수를 이용한 수치화 방법이기 때문에 단어의 의미를 고려하지 못한다는 단점이 있었습니다. (이를 토픽 모델링 관점에서는 단어의 토픽을 고려하지 못한다고도 합니다.) 이를 위한 대안으로 DTM의 잠재된(Latent) 의미를 이끌어내는 방법으로 잠재 의미 분석(Latent Semantic Analysis, LSA)이라는 방법이 있습니다. 잠재 의미 분석(Latent Semantic Indexing, LSI)이라고 부르기도 합니다. 이하 LSA라고 명명하겠습니다.

이 방법을 이해하기 위해서는 선형대수학의 특이값 분해(Singular Value Decomposition, SVD)를 이해할 필요가 있습니다. 이하 이를 SVD라고 명명하겠습니다. 이 챕터에서는 SVD를 수행하는 구체적인 선형대수학에 대해서는 설명하지 않고, **SVD가 갖고있는 의미를 이해하는 것에 초점을 맞춥니다.**

## 1.1 특이값 분해(Singular Value Decomposition, SVD)
시작하기 앞서, 여기서의 특이값 분해(Singular Value Decomposition, SVD)는 실수 벡터 공간에 한정하여 내용을 설명함을 명시합니다. SVD란 A가 m × n 행렬일 때, 다음과 같이 3개의 행렬의 곱으로 분해(decomposition)하는 것을 말합니다.

$$ A=UΣV^\text{T} $$




여기서 각 3개의 행렬은 다음과 같은 조건을 만족합니다.

$ U: m × m\ \text{직교행렬}\ (AA^\text{T}=U(ΣΣ^\text{T})U^\text{T}) $  
$ V: n × n\ \text{직교행렬}\ (A^\text{T}A=V(Σ^\text{T}Σ)V^\text{T}) $  
$ Σ: m × n\ \text{직사각 대각행렬} $

여기서 직교행렬(orthogonal matrix)이란 자신과 자신의 전치 행렬(transposed matrix)의 곱 또는 이를 반대로 곱한 결과가 단위행렬(identity matrix)이 되는 행렬을 말합니다. 또한 대각행렬(diagonal matrix)이란 주대각선을 제외한 곳의 원소가 모두 0인 행렬을 의미합니다.

이때 SVD로 나온 대각 행렬의 대각 원소의 값을 행렬 A의 특이값(singular value)라고 합니다. 많은 용어가 한꺼번에 나와서 복잡해보이는데 차근, 차근 용어를 정리해보도록 하겠습니다.

### 1.1.1 전치 행렬(Transposed Matrix)
전치 행렬(transposed matrix)은 원래의 행렬에서 행과 열을 바꾼 행렬입니다. 즉, 주대각선을 축으로 반사 대칭을 하여 얻는 행렬입니다. 기호는 기존 행렬 표현의 우측 위에 $T$를 붙입니다. 예를 들어서 기존의 행렬을 $M$이라고 한다면, 전치 행렬은 $M^T$와 같이 표현합니다.

$ M = 
\left[
    \begin{array}{c}
      1\ 2\\
      3\ 4\\
      5\ 6\\
    \end{array}
  \right]
\ \ \ \ $ $ M^\text{T} = 
\left[
    \begin{array}{c}
      1\ 3\ 5\\
      2\ 4\ 6\\
    \end{array}
  \right]
\ \ \ \ $

### 1.1.2 단위 행렬(Identity Matrix)
단위 행렬(identity matrix)은 주대각선의 원소가 모두 1이며 나머지 원소는 모두 0인 정사각 행렬을 말합니다. 보통 줄여서 대문자 $I$로 표현하기도 하는데, 2 × 2 단위 행렬과 3 × 3 단위 행렬을 표현해보면 다음과 같습니다.

$ I = 
\left[
    \begin{array}{c}
      1\ 0\\
      0\ 1\\
    \end{array}
  \right]
\ \ \ \ $$ I = 
\left[
    \begin{array}{c}
      1\ 0\ 0\\
      0\ 1\ 0\\
      0\ 0\ 1\\
    \end{array}
  \right]
\ \ \ \ $

### 1.1.3 역행렬(Inverse Matrix)
단위 행렬(identity matrix)를 이해했다면 역행렬(inverse matrix)을 정의할 수 있습니다. 만약 행렬 $A$와 어떤 행렬을 곱했을 때, 결과로서 단위 행렬이 나온다면 이때의 어떤 행렬을 $A$의 역행렬이라고 하며, $A^{-1}$라고 표현합니다.

$$ A\ ×\ A^{-1} = I $$

$ \left[
    \begin{array}{c}
      1\ 2\ 3\\
      4\ 5\ 6\\
      7\ 8\ 9\\
    \end{array}
  \right]
×
\left[
    \begin{array}{c}
      \ \ \ \ \ \ \ \ \\
      \ \ \ \ ?\ \ \ \\
      \ \ \ \ \ \ \ \ \\
    \end{array}
  \right]
=
\left[
    \begin{array}{c}
      1\ 0\ 0\\
      0\ 1\ 0\\
      0\ 0\ 1\\
    \end{array}
  \right] $

### 1.1.4 직교 행렬(Orthogonal matrix)
다시 직교 행렬(orthogonal matrix)의 정의로 돌아가서, 실수 행렬 $A$에 대해서 $A$ X $A^T = I$를 만족하면서 $A^T$ X $A = I$을 만족하는 행렬 를 직교 행렬이라고 합니다. 그런데 역행렬의 정의를 다시 생각해보면, 결국 직교 행렬은 $A^{-1} = A^T$를 만족합니다.

### 1.1.5 대각 행렬(Diagonal matrix)
대각행렬(diagonal matrix)은 주대각선을 제외한 곳의 원소가 모두 0인 행렬을 말합니다. 아래의 그림에서는 주대각선의 원소를 $a$라고 표현하고 있습니다. 만약 대각 행렬 $Σ$가 3 × 3 행렬이라면, 다음과 같은 모양을 가집니다.

$ Σ=
\left[
    \begin{array}{c}
      a\ \ 0\ \ 0\\
      0\ \ a\ \ 0\\
      0\ \ 0\ \ a\\
    \end{array}
  \right] $
  
여기까진 정사각 행렬이기 때문에 직관적으로 이해가 쉽습니다. 그런데 정사각 행렬이 아니라 직사각 행렬이 될 경우를 잘 보아야 헷갈리지 않습니다. 만약 행의 크기가 열의 크기보다 크다면 다음과 같은 모양을 가집니다. 즉, m × n 행렬일 때, m > n인 경우입니다.

$ Σ=
\left[
    \begin{array}{c}
      a\ \ 0\ \ 0\\
      0\ \ a\ \ 0\\
      0\ \ 0\ \ a\\
      0\ \ 0\ \ 0\\
    \end{array}
  \right] $
  
  
반면 n > m인 경우에는 다음과 같은 모양을 가집니다.

$ Σ=
\left[
    \begin{array}{c}
      a\ \ 0\ \ 0\ \ 0\\
      0\ \ a\ \ 0\ \ 0\\
      0\ \ 0\ \ a\ \ 0\\
    \end{array}
  \right] $
  
  
여기까지는 일반적인 대각 행렬에 대한 정의입니다. SVD를 통해 나온 대각 행렬 Σ는 추가적인 성질을 가지는데, 대각 행렬 $Σ$의 주대각원소를 행렬 $A$의 특이값(singular value)라고 하며, 이를 $\sigma_{1}, \sigma_{2} ... \sigma_{r}$라고 표현한다고 하였을 때 특이값 $\sigma_{1}, \sigma_{2} ... \sigma_{r}$은 내림차순으로 정렬되어 있다는 특징을 가집니다.

아래의 그림은 특이값 12.4, 9.5, 1.3이 내림차순으로 정렬되어져 있는 모습을 보여줍니다.

$ Σ=
\left[
    \begin{array}{c}
      12.4\ \ 0\ \ 0\\
      0\ \ 9.5\ \ 0\\
      0\ \ 0\ \ 1.3\\
    \end{array}
  \right] $


  

## 1.2 절단된 SVD(Truncated SVD)
위에서 설명한 SVD를 풀 SVD(full SVD)라고 합니다. 하지만 LSA의 경우 풀 SVD에서 나온 3개의 행렬에서 일부 벡터들을 삭제시킨 절단된 SVD(truncated SVD)를 사용하게 됩니다. 그림을 통해 이해해보도록 하겠습니다.


![python image2](https://wikidocs.net/images/page/24949/svd%EC%99%80truncatedsvd.PNG)


절단된 SVD는 대각 행렬 Σ의 대각 원소의 값 중에서 상위값 t개만 남게 됩니다. 절단된 SVD를 수행하면 값의 손실이 일어나므로 기존의 행렬 A를 복구할 수 없습니다. 또한, U행렬과 V행렬의 t열까지만 남깁니다. 여기서 t는 우리가 찾고자하는 토픽의 수를 반영한 하이퍼파라미터값입니다. 하이퍼파라미터란 사용자가 직접 값을 선택하며 성능에 영향을 주는 매개변수를 말합니다. t를 선택하는 것은 쉽지 않은 일입니다. t를 크게 잡으면 기존의 행렬 A로부터 다양한 의미를 가져갈 수 있지만, t를 작게 잡아야만 노이즈를 제거할 수 있기 때문입니다.

이렇게 일부 벡터들을 삭제하는 것을 데이터의 차원을 줄인다고도 말하는데, 데이터의 차원을 줄이게되면 당연히 풀 SVD를 하였을 때보다 직관적으로 계산 비용이 낮아지는 효과를 얻을 수 있습니다.

하지만 계산 비용이 낮아지는 것 외에도 상대적으로 중요하지 않은 정보를 삭제하는 효과를 갖고 있는데, 이는 영상 처리 분야에서는 노이즈를 제거한다는 의미를 갖고 **자연어 처리 분야에서는 설명력이 낮은 정보를 삭제하고 설명력이 높은 정보를 남긴다는 의미**를 갖고 있습니다. 즉, 다시 말하면 **기존의 행렬에서는 드러나지 않았던 심층적인 의미를 확인할 수 있게 해줍니다.(= 잠재된 의미)**

## 1.3 잠재 의미 분석(Latent Semantic Analysis, LSA)
기존의 DTM이나 DTM에 단어의 중요도에 따른 가중치를 주었던 TF-IDF 행렬은 단어의 의미를 전혀 고려하지 못한다는 단점을 갖고 있었습니다. LSA는 기본적으로 DTM이나 TF-IDF 행렬에 절단된 SVD(truncated SVD)를 사용하여 차원을 축소시키고, 단어들의 잠재적인 의미를 끌어낸다는 아이디어를 갖고 있습니다.

실습을 통해서 이해해보도록 하겠습니다.

In [1]:
import pandas as pd
tmp = pd.DataFrame({"문서1":{"과일이":0
                     ,"길고":0
                     ,"노란":0
                     ,"먹고":1
                     ,"바나나":0
                     ,"사과":1
                     ,"싶은":1
                     ,"저는":0
                     ,"좋아요":0
                    }
            , "문서2":{"과일이":0
                     ,"길고":0
                     ,"노란":0
                     ,"먹고":1
                     ,"바나나":1
                     ,"사과":0
                     ,"싶은":1
                     ,"저는":0
                     ,"좋아요":0
                    }
           , "문서3":{"과일이":0
                     ,"길고":1
                     ,"노란":1
                     ,"먹고":0
                     ,"바나나":2
                     ,"사과":0
                     ,"싶은":0
                     ,"저는":0
                     ,"좋아요":0
                    }
          , "문서4":{"과일이":1
                     ,"길고":0
                     ,"노란":0
                     ,"먹고":0
                     ,"바나나":0
                     ,"사과":0
                     ,"싶은":0
                     ,"저는":1
                     ,"좋아요":1
                    }}).transpose()

tmp

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
문서1,0,0,0,1,0,1,1,0,0
문서2,0,0,0,1,1,0,1,0,0
문서3,0,1,1,0,2,0,0,0,0
문서4,1,0,0,0,0,0,0,1,1


위와 같은 DTM을 실제로 파이썬을 통해서 만들면 다음과 같습니다.

In [5]:
import numpy as np
A=np.array([[0,0,0,1,0,1,1,0,0],[0,0,0,1,1,0,1,0,0],[0,1,1,0,2,0,0,0,0],[1,0,0,0,0,0,0,1,1]])
print(np.shape(A))
print(pd.DataFrame(A))


(4, 9)
   0  1  2  3  4  5  6  7  8
0  0  0  0  1  0  1  1  0  0
1  0  0  0  1  1  0  1  0  0
2  0  1  1  0  2  0  0  0  0
3  1  0  0  0  0  0  0  1  1


4 × 9의 크기를 가지는 DTM이 생성되었습니다. 이에 대해서 풀 SVD(full SVD)를 수행해보겠습니다. 단, 여기서는 대각 행렬의 변수명을 Σ가 아니라 S를 사용합니다. 또한 V의 전치 행렬을 VT라고 하겠습니다. 



In [6]:
# A행렬(DTM)을 full 방식으로 SVD(U, S, VT)
U, s, VT = np.linalg.svd(A, full_matrices = True)

In [9]:
print(U.round(2))
print(np.shape(U))


[[-0.24  0.75  0.   -0.62]
 [-0.51  0.44 -0.    0.74]
 [-0.83 -0.49 -0.   -0.27]
 [-0.   -0.    1.    0.  ]]
(4, 4)


4 × 4의 크기를 가지는 직교 행렬 U가 생성되었습니다. 이제 대각 행렬 S를 확인해봅시다.



In [10]:
print(s.round(2))
np.shape(s)

[2.69 2.05 1.73 0.77]


(4,)

Numpy의 linalg.svd()는 **특이값 분해의 결과로 대각 행렬이 아니라 특이값의 리스트를 반환**합니다. 그러므로 앞서 본 수식의 형식으로 보려면 이를 다시 대각 행렬로 바꾸어 주어야 합니다. 우선 특이값을 s에 저장하고 대각 행렬 크기의 행렬을 생성한 후에 그 행렬에 특이값을 삽입해도록 하겠습니다.



In [11]:
S = np.zeros((4, 9)) # 대각 행렬의 크기인 4 x 9의 임의의 행렬 생성
S[:4, :4] = np.diag(s) # 특이값을 대각행렬에 삽입
print(S.round(2))
np.shape(S)

[[2.69 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   2.05 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   1.73 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.77 0.   0.   0.   0.   0.  ]]


(4, 9)

4 × 9의 크기를 가지는 대각 행렬 S가 생성되었습니다. 2.69 > 2.05 > 1.73 > 0.77 순으로 값이 내림차순을 보이는 것을 확인할 수 있습니다.



In [12]:
print(VT.round(2))
np.shape(VT)

[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]
 [ 0.58 -0.    0.    0.   -0.    0.   -0.    0.58  0.58]
 [ 0.   -0.35 -0.35  0.16  0.25 -0.8   0.16 -0.   -0.  ]
 [-0.   -0.78 -0.01 -0.2   0.4   0.4  -0.2   0.    0.  ]
 [-0.29  0.31 -0.78 -0.24  0.23  0.23  0.01  0.14  0.14]
 [-0.29 -0.1   0.26 -0.59 -0.08 -0.08  0.66  0.14  0.14]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19  0.75 -0.25]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19 -0.25  0.75]]


(9, 9)

9 × 9의 크기를 가지는 직교 행렬 VT(V의 전치 행렬)가 생성되었습니다. 즉, U × S × VT를 하면 기존의 행렬 A가 나와야 합니다. Numpy의 allclose()는 2개의 행렬이 동일하면 True를 리턴합니다. 이를 사용하여 정말로 기존의 행렬 A와 동일한지 확인해보겠습니다.

U.shape = 4, 4  
S.shape = 4, 9  
VT.shape = 9, 9



In [13]:
np.allclose(A, np.dot(np.dot(U,S), VT).round(2))

True

In [18]:
A

array([[0, 0, 0, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 1, 1, 0, 1, 0, 0],
       [0, 1, 1, 0, 2, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 1]])

지금까지 수행한 것은 풀 SVD(Full SVD)입니다. 이제 t를 정하고, 절단된 SVD(Truncated SVD)를 수행해보도록 합시다. 여기서는 t=2로 하겠습니다. 우선 대각 행렬 S 내의 특이값 중에서 상위 2개만 남기고 제거해보도록 하겠습니다.



In [36]:
S=S[:2,:2]
print(S.round(2))

[[2.69 0.  ]
 [0.   2.05]]


상위 2개의 값만 남기고 나머지는 모두 제거된 것을 볼 수 있습니다. 이제 직교 행렬 U에 대해서도 2개의 열만 남기고 제거합니다.(asis 4 * 2)



In [37]:
U=U[:,:2]
print(U.round(2))

[[-0.24  0.75]
 [-0.51  0.44]
 [-0.83 -0.49]
 [-0.   -0.  ]]


2개의 열만 남기고 모두 제거가 된 것을 볼 수 있습니다. 이제 행렬 V의 전치 행렬인 VT에 대해서 2개의 행(= 2 * 9)만 남기고 제거합니다. 이는 V관점에서는 2개의 열만 남기고 제거한 것이 됩니다.(asis 4 * 9)



In [38]:
VT=VT[:2,:]
print(VT.round(2))


[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]]


이제 축소된 행렬 U, S, VT에 대해서 다시 U × S × VT연산을 하면 기존의 A와는 다른 결과가 나오게 됩니다. **값이 손실되었기 때문에 이 세 개의 행렬로는 이제 기존의 A행렬을 복구할 수 없습니다.** U × S × VT연산을 해서 나오는 값을 A_prime이라 하고 기존의 행렬 A와 값을 비교해보도록 하겠습니다.



In [41]:
A_prime=np.dot(np.dot(U,S), VT)
print(A)
print(A_prime.round(2))

[[0 0 0 1 0 1 1 0 0]
 [0 0 0 1 1 0 1 0 0]
 [0 1 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 0 1 1]]
[[ 0.   -0.17 -0.17  1.08  0.12  0.62  1.08 -0.   -0.  ]
 [ 0.    0.2   0.2   0.91  0.86  0.45  0.91  0.    0.  ]
 [ 0.    0.93  0.93  0.03  2.05 -0.17  0.03  0.    0.  ]
 [ 0.    0.    0.    0.    0.   -0.    0.    0.    0.  ]]


대체적으로 기존에 0인 값들은 0에 가가운 값이 나오고, 1인 값들은 1에 가까운 값이 나오는 것을 볼 수 있습니다. 또한 값이 제대로 복구되지 않은 구간도 존재해보입니다. 이제 이렇게 차원이 축소된 U, S, VT의 크기가 어떤 의미를 가지고 있는지 알아봅시다.

축소된 U는 4 × 2의 크기를 가지는데, 이는 잘 생각해보면 **문서의 개수 × 토픽의 수 t의 크기**입니다. **단어의 개수인 9는 유지되지 않는데 문서의 개수인 4의 크기가 유지되었으니 4개의 문서 각각을 2개의 값으로 표현**하고 있습니다. 즉, **U의 각 행은 잠재 의미를 표현하기 위한 수치화 된 각각의 문서 벡터**라고 볼 수 있습니다. 축소된 VT는 2 × 9의 크기를 가지는데, 이는 잘 생각해보면 **토픽의 수 t × 단어의 개수의 크기**입니다. VT의 각 열은 **잠재 의미를 표현하기 위해 수치화된 각각의 단어 벡터**라고 볼 수 있습니다.

이 문서 벡터들과 단어 벡터들을 통해 다른 문서의 유사도, 다른 단어의 유사도, 단어(쿼리)로부터 문서의 유사도를 구하는 것들이 가능해집니다.

## 1.4 실습을 통한 이해
사이킷런에서는 Twenty Newsgroups이라고 불리는 20개의 다른 주제를 가진 뉴스그룹 데이터를 제공합니다. 앞서 언급했듯이 LSA가 토픽 모델링에 최적화 된 알고리즘은 아니지만, 토픽 모델링이라는 분야의 시초가 되는 알고리즘입니다. 여기서는 LSA를 사용해서 문서의 수를 원하는 토픽의 수로 압축한 뒤에 각 토픽당 가장 중요한 단어 5개를 출력하는 실습으로 토픽 모델링을 수행합니다.

### 1.4.1 뉴스그룹 데이터에 대한 이해

In [43]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
len(documents)

11314

훈련에 사용할 뉴스그룹 데이터는 총 11,314개입니다. 이 중 첫번째 훈련용 샘플을 출력해보겠습니다.



In [48]:
documents[1]

"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim.  And I'm sorry that you have these feelings of\ndenial about the faith you need to get by.  Oh well, just pretend that it will\nall end happily ever after anyway.  Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) \n--\nBake Timmons, III"

뉴스그룹 데이터에는 특수문자가 포함된 다수의 영어문장으로 구성되어져 있습니다. 이런 형식의 샘플이 총 11,314개 존재합니다. 사이킷런이 제공하는 뉴스그룹 데이터에서 target_name에는 본래 이 뉴스그룹 데이터가 어떤 20개의 카테고리를 갖고있었는지가 저장되어져 있습니다. 이를 출력해보겠습니다.

In [49]:
print(dataset.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


### 1.4.2 텍스트 전처리
시작하기 앞서, 텍스트 데이터에 대해서 가능한한 정제 과정을 거쳐야만 합니다. 기본적인 아이디어는 알파벳을 제외한 구두점, 숫자, 특수 문자를 제거하는 것입니다. 이는 텍스트 전처리 챕터에서 정제 기법으로 배웠던 정규 표현식을 통해서 해결할 수 있습니다. 또한 짧은 단어는 유용한 정보를 담고있지 않다고 가정하고, 길이가 짧은 단어도 제거합니다. 그리고 마지막으로 모든 알파벳을 소문자로 바꿔서 단어의 개수를 줄이는 작업을 합니다.

In [50]:
news_df = pd.DataFrame({'document':documents})

In [62]:
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

<ipython-input-62-b4124dfb5e6a>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")


데이터의 정제가 끝났습니다. 다시 첫번째 훈련용 뉴스그룹 샘플만 출력하여 정제 전, 후에 어떤 차이가 있는지 확인해보겠습니다.

In [65]:
news_df.head(5)

,document,clean_doc
0,Well i'm not sure about the story nad it did s...,well sure about story seem biased what disagre...
1,"\n\n\n\n\n\n\nYeah, do you expect people to re...",yeah expect people read actually accept hard a...
2,Although I realize that principle is not one o...,although realize that principle your strongest...
3,Notwithstanding all the legitimate fuss about ...,notwithstanding legitimate fuss about this pro...
4,"Well, I will have to change the scoring on my ...",well will have change scoring playoff pool unf...


우선 특수문자가 제거되었으며, if나 you와 같은 길이가 3이하인 단어가 제거된 것을 확인할 수 있습니다. 뿐만 아니라 대문자가 전부 소문자로 바뀌었습니다. 이제 뉴스그룹 데이터에서 불용어를 제거합니다. 불용어를 제거하기 위해서 토큰화를 우선 수행합니다. 토큰화와 불용어 제거를 순차적으로 진행합니다.



In [66]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english') # NLTK로부터 불용어를 받아옵니다.
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
# 불용어를 제거합니다.

다시 첫번째 훈련용 뉴스그룹 샘플을 출력합니다.



In [67]:
print(tokenized_doc[1])


['yeah', 'expect', 'people', 'read', 'actually', 'accept', 'hard', 'atheism', 'need', 'little', 'leap', 'faith', 'jimmy', 'logic', 'runs', 'steam', 'sorry', 'pity', 'sorry', 'feelings', 'denial', 'faith', 'need', 'well', 'pretend', 'happily', 'ever', 'anyway', 'maybe', 'start', 'newsgroup', 'atheist', 'hard', 'bummin', 'much', 'forget', 'flintstone', 'chewables', 'bake', 'timmons']


기존에 있었던 불용어에 속하던 your, about, just, that, will, after 단어들이 사라졌을 뿐만 아니라, 토큰화가 수행된 것을 확인할 수 있습니다.



In [74]:
# regex만 실행했던 결과
news_df.clean_doc[1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy your logic runs steam sorry pity sorry that have these feelings denial about faith need well just pretend that will happily ever after anyway maybe start newsgroup atheist hard bummin much forget your flintstone chewables bake timmons'

### 1.4.3 TF-IDF 행렬 만들기
불용어 제거를 위해 토큰화 작업을 수행하였지만, TfidfVectorizer(TF-IDF 챕터 참고)는 기본적으로 토큰화가 되어있지 않은 텍스트 데이터를 입력으로 사용합니다. 그렇기 때문에 TfidfVectorizer를 사용해서 TF-IDF 행렬을 만들기 위해서 다시 토큰화 작업을 역으로 취소하는 작업을 수행해보도록 하겠습니다. 이를 역토큰화(Detokenization)라고 합니다.

In [75]:
# 역토큰화 (토큰화 작업을 역으로 되돌림)
detokenized_doc = []
for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

news_df['clean_doc'] = detokenized_doc

news_df["clean_doc"][1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy logic runs steam sorry pity sorry feelings denial faith need well pretend happily ever anyway maybe start newsgroup atheist hard bummin much forget flintstone chewables bake timmons'

정상적으로 불용어가 제거된 상태에서 역토큰화가 수행되었음을 확인할 수 있습니다.

이제 사이킷런의 TfidfVectorizer를 통해 단어 1,000개에 대한 TF-IDF 행렬을 만들 것입니다. 물론 텍스트 데이터에 있는 모든 단어를 가지고 행렬을 만들 수는 있겠지만, 여기서는 1,000개의 단어로 제한하도록 하겠습니다.

In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000, # 상위 1,000개의 단어를 보존 
max_df = 0.5, 
smooth_idf=True)

X = vectorizer.fit_transform(news_df['clean_doc'])
X.shape # TF-IDF 행렬의 크기 확인

(11314, 1000)

11,314 × 1,000의 크기를 가진 TF-IDF 행렬이 생성되었음을 확인할 수 있습니다.



### 1.4.4 토픽 모델링(Topic Modeling)
이제 TF-IDF 행렬을 다수의 행렬로 분해해보도록 하겠습니다. 여기서는 사이킷런의 절단된 SVD(Truncated SVD)를 사용합니다. 절단된 SVD를 사용하면 차원을 축소할 수 있습니다. 원래 기존 뉴스그룹 데이터가 20개의 카테고리를 갖고있었기 때문에, 20개의 토픽을 가졌다고 가정하고 토픽 모델링을 시도해보겠습니다. 토픽의 숫자는 n_components의 파라미터로 지정이 가능합니다.

In [79]:
from sklearn.decomposition import TruncatedSVD
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(X)
len(svd_model.components_)

20

여기서 svd_model.componets_는 앞서 배운 LSA에서 VT에 해당됩니다.



In [80]:
np.shape(svd_model.components_)


(20, 1000)

정확하게 토픽의 수 t × 단어의 수의 크기를 가지는 것을 볼 수 있습니다.



In [88]:
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(svd_model.components_,terms)

Topic 1: [('like', 0.21386), ('know', 0.20046), ('people', 0.19293), ('think', 0.17805), ('good', 0.15128)]
Topic 2: [('thanks', 0.32888), ('windows', 0.29088), ('card', 0.18069), ('drive', 0.17455), ('mail', 0.15111)]
Topic 3: [('game', 0.37064), ('team', 0.32443), ('year', 0.28154), ('games', 0.2537), ('season', 0.18419)]
Topic 4: [('drive', 0.53324), ('scsi', 0.20165), ('hard', 0.15628), ('disk', 0.15578), ('card', 0.13994)]
Topic 5: [('windows', 0.40399), ('file', 0.25436), ('window', 0.18044), ('files', 0.16078), ('program', 0.13894)]
Topic 6: [('chip', 0.16114), ('government', 0.16009), ('mail', 0.15625), ('space', 0.1507), ('information', 0.13562)]
Topic 7: [('like', 0.67086), ('bike', 0.14236), ('chip', 0.11169), ('know', 0.11139), ('sounds', 0.10371)]
Topic 8: [('card', 0.46633), ('video', 0.22137), ('sale', 0.21266), ('monitor', 0.15463), ('offer', 0.14643)]
Topic 9: [('know', 0.46047), ('card', 0.33605), ('chip', 0.17558), ('government', 0.1522), ('video', 0.14356)]
Topic 10

## 1.5 LSA의 장단점(Pros and Cons of LSA)
정리해보면 LSA는 쉽고 빠르게 구현이 가능할 뿐만 아니라 단어의 잠재적인 의미를 이끌어낼 수 있어 문서의 유사도 계산 등에서 좋은 성능을 보여준다는 장점을 갖고 있습니다. 하지만 SVD의 특성상 이미 계산된 LSA에 새로운 데이터를 추가하여 계산하려고하면 보통 처음부터 다시 계산해야 합니다. 즉, 새로운 정보에 대해 업데이트가 어렵습니다.(=본래의 A행렬에 행이나 열이 추가되면 SVD의 결과 역시 달라질 수 밖에 없으니)  이는 최근 LSA 대신 Word2Vec 등 단어의 의미를 벡터화할 수 있는 또 다른 방법론인 인공 신경망 기반의 방법론이 각광받는 이유이기도 합니다.



https://sragent.tistory.com/entry/Latent-Semantic-AnalysisLSA  
https://bskyvision.com/251  
https://simonpaarlberg.com/post/latent-semantic-analyses/  
  
LSA로 텍스트 분류하기 :  
https://seantrottcom.wordpress.com/2018/08/20/tutorial-document-classification-with-latent-semantic-analysis-lsa/

# 2. 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA)
토픽 모델링은 문서의 집합에서 토픽을 찾아내는 프로세스를 말합니다. 이는 검색 엔진, 고객 민원 시스템 등과 같이 문서의 주제를 알아내는 일이 중요한 곳에서 사용됩니다. 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA)은 토픽 모델링의 대표적인 알고리즘입니다. 줄여서 LDA라고 합니다.

LDA는 문서들은 토픽들의 혼합으로 구성되어져 있으며, 토픽들은 확률 분포에 기반하여 단어들을 생성한다고 가정합니다. 데이터가 주어지면, LDA는 문서가 생성되던 과정을 역추적합니다.

참고 링크 : https://lettier.com/projects/lda-topic-modeling/

위의 사이트는 코드 작성 없이 입력한 문서들로부터 DTM을 만들고 LDA를 수행한 결과를 보여주는 웹 사이트입니다.

## 2.1 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA) 개요
우선 LDA의 내부 메커니즘에 대해서 이해하기 전에, LDA를 일종의 블랙 박스로 보고 LDA에 문서 집합을 입력하면, 어떤 결과를 보여주는지 간소화 된 예를 들어 보겠습니다. 아래와 같은 3개의 문서가 있다고 합시다. 지금의 예제는 간단해서 눈으로도 토픽 모델링을 할 수 있을 것 같지만, 실제 수십만개 이상의 문서가 있는 경우는 직접 토픽을 찾아내는 것이 어렵기 때문에 LDA의 도움이 필요합니다.

**문서1 : 저는 사과랑 바나나를 먹어요  
문서2 : 우리는 귀여운 강아지가 좋아요  
문서3 : 저의 깜찍하고 귀여운 강아지가 바나나를 먹어요**

LDA를 수행할 때 문서 집합에서 토픽이 몇 개가 존재할지 가정하는 것은 사용자가 해야 할 일입니다. 여기서는 LDA에 2개의 토픽을 찾으라고 요청하겠습니다. 토픽의 개수를 의미하는 변수를 k라고 하였을 때, k를 2로 한다는 의미입니다. k의 값을 잘못 선택하면 원치않는 이상한 결과가 나올 수 있습니다. 이렇게 **모델의 성능에 영향을 주는 사용자가 직접 선택하는 매개변수를 머신 러닝 용어로 하이퍼파라미터**라고 합니다. 이러한 하이퍼파라미터의 선택은 여러 실험을 통해 얻은 값일 수도 있고, 우선 시도해보는 값일 수도 있습니다.

LDA가 위의 세 문서로부터 2개의 토픽을 찾은 결과는 아래와 같습니다. 여기서는 LDA 입력 전에 주어와 불필요한 조사 등을 제거하는 전처리 과정은 거쳤다고 가정합니다. 즉, **전처리 과정을 거친 DTM이 LDA의 입력이 되었다고 가정**합니다.

LDA는 각 문서의 토픽 분포와 각 토픽 내의 단어 분포를 추정합니다.

**<각 문서의 토픽 분포>**  
문서1 : 토픽 A 100%  
문서2 : 토픽 B 100%  
문서3 : 토픽 B 60%, 토픽 A 40%
  
**<각 토픽의 단어 분포>**  
토픽A : 사과 20%, 바나나 40%, 먹어요 40%, 귀여운 0%, 강아지 0%, 깜찍하고 0%, 좋아요 0%  
토픽B : 사과 0%, 바나나 0%, 먹어요 0%, 귀여운 33%, 강아지 33%, 깜찍하고 16%, 좋아요 16%  

LDA는 토픽의 제목을 정해주지 않지만, 이 시점에서 알고리즘의 사용자는 위 결과로부터 두 토픽이 각각 과일에 대한 토픽과 강아지에 대한 토픽이라고 판단해볼 수 있습니다. 이제 LDA에 대해서 알아봅시다. 실제로 LDA는 아래의 설명보다 훨씬 더 복잡하지만, 여기서는 수학적인 수식은 배제하고 개념적 이해에 초점을 둡니다.

## 2.2 LDA의 가정
LDA는 문서의 집합으로부터 어떤 토픽이 존재하는지를 알아내기 위한 알고리즘입니다. LDA는 앞서 배운 빈도수 기반의 표현 방법인 BoW의 행렬 **DTM 또는 TF-IDF 행렬을 입력**으로 하는데, 이로부터 알 수 있는 사실은 **LDA는 단어의 순서는 신경쓰지 않겠다**는 겁니다.

LDA는 문서들로부터 토픽을 뽑아내기 위해서 이러한 가정을 염두해두고 있습니다. 모든 문서 하나, 하나가 작성될 때 그 문서의 작성자는 이러한 생각을 했습니다. '나는 이 문서를 작성하기 위해서 이런 주제들을 넣을거고, 이런 주제들을 위해서는 이런 단어들을 넣을 거야.' 조금 더 구체적으로 알아보겠습니다. 각각의 문서는 다음과 같은 과정을 거쳐서 작성되었다고 가정합니다.

1) 문서에 사용할 단어의 개수 N을 정합니다.  
     - Ex) 5개의 단어를 정하였습니다.  
2) 문서에 사용할 토픽의 혼합을 확률 분포에 기반하여 결정합니다.  
     - Ex) 위 예제와 같이 토픽이 2개라고 하였을 때 강아지 토픽을 60%, 과일 토픽을 40%와 같이 선택할 수 있습니다.  
3) 문서에 사용할 각 단어를 (아래와 같이) 정합니다.
- 3-1) 토픽 분포에서 토픽 T를 확률적으로 고릅니다. 
    - Ex) 60% 확률로 강아지 토픽을 선택하고, 40% 확률로 과일 토픽을 선택할 수 있습니다.  
- 3-2) 선택한 토픽 T에서 단어의 출현 확률 분포에 기반해 문서에 사용할 단어를 고릅니다.  
    - Ex) 강아지 토픽을 선택하였다면, 33% 확률로 강아지란 단어를 선택할 수 있습니다. 이제 3)을 반복하면서 문서를 완성합니다.  

이러한 과정을 통해 문서가 작성되었다는 가정 하에 LDA는 토픽을 뽑아내기 위하여 위 과정을 역으로 추적하는 역공학(reverse engneering)을 수행합니다.

## 2.3 LDA의 수행하기
이제 LDA의 수행 과정을 정리해보겠습니다.

1) 사용자는 알고리즘에게 토픽의 개수 k를 알려줍니다.  
앞서 말하였듯이 LDA에게 토픽의 개수를 알려주는 역할은 사용자의 역할입니다. LDA는 토픽의 개수 k를 입력받으면, k개의 토픽이 M개의 전체 문서에 걸쳐 분포되어 있다고 가정합니다.  

2) 모든 단어를 k개 중 하나의 토픽에 할당합니다.  
이제 LDA는 모든 문서의 모든 단어에 대해서 k개 중 하나의 토픽을 랜덤으로 할당합니다. 이 작업이 끝나면 각 문서는 토픽을 가지며, 토픽은 단어 분포를 가지는 상태입니다. 물론 랜덤으로 할당하였기 때문에 사실 이 결과는 전부 틀린 상태입니다. 만약 한 단어가 한 문서에서 2회 이상 등장하였다면, 각 단어는 서로 다른 토픽에 할당되었을 수도 있습니다.  

3) 이제 모든 문서의 모든 단어에 대해서 아래의 사항을 반복 진행합니다. (iterative)  

- 3-1) 어떤 문서의 각 단어 w는 자신은 잘못된 토픽에 할당되어져 있지만, 다른 단어들은 전부 올바른 토픽에 할당되어져 있는 상태라고 가정합니다. 이에 따라 단어 w는 아래의 두 가지 기준에 따라서 토픽이 재할당됩니다.
    - p(topic t | document d) : 문서 d의 단어들 중 토픽 t에 해당하는 단어들의 비율
    - p(word w | topic t) : 각 토픽들 t에서 해당 단어 w의 분포

이를 반복하면, 모든 할당이 완료된 수렴 상태가 됩니다. 두 가지 기준이 어떤 의미인지 예를 들어보겠습니다. 설명의 편의를 위해서 두 개의 문서라는 새로운 예를 사용합니다.

![python image2](https://wikidocs.net/images/page/30708/lda1.PNG)

위의 그림은 두 개의 문서 doc1과 doc2를 보여줍니다. 여기서는 doc1의 세번째 단어 apple의 토픽을 결정하고자 합니다.



![python image2](https://wikidocs.net/images/page/30708/lda3.PNG)

우선 첫번째로 사용하는 기준은 문서 doc1의 단어들이 어떤 토픽에 해당하는지를 봅니다. doc1의 모든 단어들은 토픽 A와 토픽 B에 50 대 50의 비율로 할당되어져 있으므로, 이 기준에 따르면 단어 apple은 토픽 A 또는 토픽 B 둘 중 어디에도 속할 가능성이 있습니다.



![python image2](https://wikidocs.net/images/page/30708/lda2.PNG)

두번째 기준은 단어 apple이 전체 문서에서 어떤 토픽에 할당되어져 있는지를 봅니다. 이 기준에 따르면 단어 apple은 토픽 B에 할당될 가능성이 높습니다. 이러한 두 가지 기준을 참고하여 LDA는 doc1의 apple을 어떤 토픽에 할당할지 결정합니다.



## 2.4 잠재 디리클레 할당과 잠재 의미 분석의 차이
LSA : DTM을 차원 축소 하여 축소 차원에서 근접 단어들을 토픽으로 묶는다.  
LDA : 단어가 특정 토픽에 존재할 확률과 문서에 특정 토픽이 존재할 확률을 결합확률로 추정하여 토픽을 추출한다.

## 2.5 실습을 통한 이해
이제 LDA를 실습을 통해 직접 진행해보도록 하겠습니다. LSA 챕터에서는 사이킷런(sklearn)을 사용하였지만, 이번 챕터에서는 gensim을 사용하므로 앞서 챕터와 실습 과정이 확연히 다릅니다. 사이킷런을 통해 LDA를 진행하는 실습은 아래의 링크에 작성하였습니다. 아래의 링크에서는 LSA 챕터의 실습과 진행 과정이 거의 유사하니 참고하시기 바랍니다.

사이킷런으로 LDA 실습 : https://wikidocs.net/40710

### 2.5.1 정수 인코딩과 단어 집합 만들기
바로 이전 챕터인 LSA 챕터에서 사용하였던 Twenty Newsgroups이라고 불리는 20개의 다른 주제를 가진 뉴스 데이터를 다시 사용합니다. 전처리 과정은 이전 챕터와 중복되므로 생략합니다. 동일한 전처리 과정을 거친 후에 tokenized_doc으로 저장한 상태라고 합시다. 훈련용 뉴스를 5개만 출력해보겠습니다.

In [90]:
tokenized_doc[:5]


0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object

이제 각 단어에 정수 인코딩을 하는 동시에, 각 뉴스에서의 단어의 빈도수를 기록해보겠습니다. 여기서는 각 단어를 (word_id, word_frequency)의 형태로 바꾸고자 합니다. word_id는 단어가 정수 인코딩된 값이고, word_frequency는 해당 뉴스에서의 해당 단어의 빈도수를 의미합니다. 이는 gensim의 corpora.Dictionary()를 사용하여 손쉽게 구할 수 있습니다. 전체 뉴스에 대해서 정수 인코딩을 수행하고, 두번째 뉴스를 출력해봅시다.



In [92]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


두번째 뉴스의 출력 결과를 봅시다. 위의 출력 결과 중에서 (66, 2)는 정수 인코딩이 66으로 할당된 단어가 두번째 뉴스에서는 두 번 등장하였음을 의미합니다. 66이라는 값을 가지는 단어가 정수 인코딩이 되기 전에는 어떤 단어였는지 확인하여봅시다. 이는 dictionary[]에 기존 단어가 무엇인지 알고자하는 정수값을 입력하여 확인할 수 있습니다.

In [93]:
print(dictionary[66])


faith


기존에는 단어 'faith'이었음을 알 수 있습니다. 총 학습된 단어의 개수를 확인해보겠습니다. 이는 dictionary의 길이를 확인하면 됩니다.



In [95]:
len(dictionary)


64281

### 2.5.2 LDA 모델 훈련시키기
기존의 뉴스 데이터가 총 20개의 카테고리를 가지고 있었으므로 토픽의 개수를 20으로 하여 LDA 모델을 학습시켜보도록 하겠습니다.

In [96]:
import gensim
NUM_TOPICS = 20 #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.016*"navy" + 0.011*"naval" + 0.010*"yankees" + 0.006*"disc"')
(1, '0.010*"homosexual" + 0.007*"grass" + 0.006*"spare" + 0.006*"exhaust"')
(2, '0.017*"play" + 0.015*"period" + 0.013*"team" + 0.012*"game"')
(3, '0.035*"jesus" + 0.022*"christian" + 0.020*"bible" + 0.017*"church"')
(4, '0.011*"armenian" + 0.011*"said" + 0.011*"people" + 0.010*"armenians"')
(5, '0.022*"file" + 0.014*"program" + 0.013*"windows" + 0.010*"files"')
(6, '0.033*"entry" + 0.025*"sale" + 0.018*"offer" + 0.017*"condition"')
(7, '0.011*"university" + 0.009*"research" + 0.009*"health" + 0.007*"medical"')
(8, '0.013*"year" + 0.010*"last" + 0.009*"good" + 0.008*"game"')
(9, '0.023*"space" + 0.008*"nasa" + 0.007*"data" + 0.005*"launch"')
(10, '0.014*"encryption" + 0.012*"chip" + 0.011*"keys" + 0.011*"clipper"')
(11, '0.051*"output" + 0.041*"window" + 0.022*"widget" + 0.016*"char"')
(12, '0.016*"wiring" + 0.011*"radar" + 0.010*"detector" + 0.009*"symbol"')
(13, '0.011*"drive" + 0.009*"like" + 0.009*"would" + 0.008*

각 단어 앞에 붙은 수치는 단어의 해당 토픽에 대한 기여도를 보여줍니다. 또한 맨 앞에 있는 토픽 번호는 0부터 시작하므로 총 20개의 토픽은 0부터 19까지의 번호가 할당되어져 있습니다. passes는 알고리즘의 동작 횟수를 말하는데, 알고리즘이 결정하는 토픽의 값이 적절히 수렴할 수 있도록 충분히 적당한 횟수를 정해주면 됩니다. 여기서는 총 15회를 수행하였습니다. 여기서는 num_words=4로 총 4개의 단어만 출력하도록 하였습니다. 만약 10개의 단어를 출력하고 싶다면 파라미터를 수정하면 됩니다.

### 2.5.3 LDA 시각화 하기
LDA 시각화를 위해서는 pyLDAvis의 설치가 필요합니다. 윈도우의 명령 프롬프트나 MAC/UNIX의 터미널에서 아래의 명령을 수행하여 pyLDAvis를 설치하시기 바랍니다.

설치가 완료되었다면 LDA 시각화 실습을 진행합니다.



In [104]:
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

C:\Users\mrjjo\anaconda3\envs\nlp38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\mrjjo\anaconda3\envs\nlp38\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


좌측의 원들은 각각의 20개의 토픽을 나타냅니다. 각 원과의 거리는 각 토픽들이 서로 얼마나 다른지를 보여줍니다. 만약 두 개의 원이 겹친다면, 이 두 개의 토픽은 유사한 토픽이라는 의미입니다. 위의 그림에서는 10번 토픽을 클릭하였고, 이에 따라 우측에는 10번 토픽에 대한 정보가 나타납니다. 한 가지 주의할 점은 LDA 모델의 출력 결과에서는 토픽 번호가 0부터 할당되어 0 ~ 19의 숫자가 사용된 것과는 달리 위의 LDA 시각화에서는 토픽의 번호가 1부터 시작하므로 각 토픽 번호는 이제 +1이 된 값인 1~20까지의 값을 가집니다.



### 2.4.4 문서 별 토픽 분포 보기
위에서 토픽 별 단어 분포는 확인하였으나, 아직 문서 별 토픽 분포에 대해서는 확인하지 못 하였습니다. 우선 문서 별 토픽 분포를 확인하는 방법을 보겠습니다. 각 문서의 토픽 분포는 이미 훈련된 LDA 모델인 ldamodel[]에 전체 데이터가 정수 인코딩 된 결과를 넣은 후에 확인이 가능합니다. 여기서는 책의 지면의 한계로 상위 5개의 문서에 대해서만 토픽 분포를 확인해보겠습니다.

In [105]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(4, 0.21487129), (7, 0.10829783), (8, 0.10703037), (9, 0.1546572), (16, 0.044215005), (18, 0.3435771), (19, 0.016843127)]
1 번째 문서의 topic 비율은 [(3, 0.15489624), (8, 0.052288078), (11, 0.02891601), (18, 0.7428298)]
2 번째 문서의 topic 비율은 [(8, 0.23474331), (15, 0.088546336), (16, 0.17177132), (18, 0.49200287)]
3 번째 문서의 topic 비율은 [(1, 0.01565064), (7, 0.13385539), (10, 0.33060008), (13, 0.12909013), (15, 0.034595374), (18, 0.34522727)]
4 번째 문서의 topic 비율은 [(2, 0.23024769), (14, 0.25502455), (18, 0.48323593)]


C:\Users\mrjjo\anaconda3\envs\nlp38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


위의 출력 결과에서 (숫자, 확률)은 각각 토픽 번호와 해당 토픽이 해당 문서에서 차지하는 분포도를 의미합니다. 예를 들어 0번째 문서의 토픽 비율에서 (4, 0.21487129)은 4번 토픽이 21.5%의 분포도를 가지는 것을 의미합니다. 위의 코드를 응용하여 좀 더 깔끔한 형태인 데이터프레임 형식으로 출력해보겠습니다.

In [106]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

C:\Users\mrjjo\anaconda3\envs\nlp38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [107]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

C:\Users\mrjjo\anaconda3\envs\nlp38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,18.0,0.3436,"[(4, 0.21486965), (7, 0.10829331), (8, 0.10702..."
1,1,18.0,0.7436,"[(3, 0.15486766), (8, 0.05158137), (11, 0.0289..."
2,2,18.0,0.4920,"[(8, 0.23471017), (15, 0.08854245), (16, 0.171..."
3,3,18.0,0.3452,"[(1, 0.015650645), (7, 0.13385467), (10, 0.330..."
4,4,18.0,0.4832,"[(2, 0.23024766), (14, 0.25502458), (18, 0.483..."
5,5,18.0,0.3133,"[(3, 0.2316008), (4, 0.23341742), (8, 0.146657..."
6,6,13.0,0.6502,"[(5, 0.08380517), (13, 0.6502454), (18, 0.2252..."
7,7,18.0,0.6990,"[(8, 0.050303496), (15, 0.0916617), (16, 0.096..."
8,8,18.0,0.3900,"[(8, 0.32762647), (9, 0.2139524), (10, 0.04484..."
9,9,13.0,0.5242,"[(3, 0.012976287), (6, 0.014931572), (9, 0.161..."


http://s-space.snu.ac.kr/bitstream/10371/95582/1/22_1_%EB%82%A8%EC%B6%98%ED%98%B8.pdf  
https://bab2min.tistory.com/568  
https://annalyzin.wordpress.com/2015/06/21/laymans-explanation-of-topic-modeling-with-lda-2/  
https://towardsdatascience.com/latent-dirichlet-allocation-15800c852699  
https://radimrehurek.com/gensim/wiki.html#latent-dirichlet-allocation  
https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/  
  
모델 저장 및 로드 하기 : https://stackabuse.com/python-for-nlp-working-with-the-gensim-library-part-2/  

전반적으로 참고하기 좋은 글 : https://shichaoji.com/2017/04/25/topicmodeling/  

동영상 강의 : https://blog.naver.com/chunjein/220946362463  

뉴스를 가지고 할 수 있는 다양한 일들 : https://www.slideshare.net/koorukuroo/20160813-pycon2016apac  